In [9]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings
import numpy as np
warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('../train/*.csv'))]

In [10]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [11]:
from sklearn.preprocessing import LabelEncoder
years = ['2017', '2018', '2019', '2020']

year_encoder = {}

for year, df in zip(years, know_train):
    print(year)
    encoders = {}
    
    for col in df.columns:
        if col == 'ID':
            continue
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = LabelEncoder()
            df[col] = df[col].map(str)
            df[col] = encoder.fit_transform(df[col])
            encoders[col] = encoder
            
            
    year_encoder[year] = encoders

2017
2018
2019
2020


In [12]:
train_data = {}
for year, df in zip(years, know_train):
    train_data[year] = {'X': df.iloc[:, 1:-1], # ID제외
                        'y': df.iloc[:, -1]} 

In [13]:
from sklearn.model_selection import train_test_split

split_data = {}
for year in tqdm(years):
    X_train,X_test,y_train,y_test = train_test_split(train_data[year]['X'],train_data[year]['y'],test_size=0.25, random_state=42)
    split_data[year] = {'X_train' : X_train,
                       'y_train' : y_train,
                       'X_test' : X_test,
                       'y_test' : y_test}

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 41.97it/s]


In [107]:
print(len(split_data[year]['X_train']))
print(len(split_data[year]['X_test']))
print(len(split_data[year]['y_train']))
print(len(split_data[year]['y_test']))

6091
2031
6091
2031


In [125]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

dt_models = {}

for year in tqdm(years):
    model = DecisionTreeClassifier(random_state=123456)
    model.fit(split_data[year]['X_train'], split_data[year]['y_train'])
    dt_models[year] = model

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.66s/it]


In [126]:
dt_f1 =[]
for year in tqdm(years):
    y_pred = dt_models[year].predict(split_data[year]['X_test'])
    dt_f1.append(f1_score(split_data[year]['y_test'],y_pred,average ='micro'))
np.mean(dt_f1)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 67.57it/s]


0.33673863361759543

In [14]:
from sklearn.ensemble import RandomForestClassifier
rf_models = {}

for year in tqdm(years):
    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(split_data[year]['X_train'], split_data[year]['y_train'])
    rf_models[year] = model

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:17<00:00,  4.25s/it]


In [16]:
from sklearn.metrics import f1_score
rf_f1 =[]
for year in tqdm(years):
    y_pred = rf_models[year].predict(split_data[year]['X_test'])
    rf_f1.append(f1_score(split_data[year]['y_test'],y_pred,average ='micro'))
np.mean(rf_f1)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.40it/s]


0.5184834483607411

In [17]:
y_pred

array([624201,  12101, 420402, ..., 621401, 135001, 307101], dtype=int64)

In [17]:
know_train[0]['aq1_1'].value_counts()

3    3116
4    3007
2    1706
5    1072
1     585
Name: aq1_1, dtype: int64

In [18]:
know_test[0].head() # 2017년도 test 샘플

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
0,0,3,4,2,2,3,3,1,0,3,...,2,26,3,497,1,1,1,3000,0,2300
1,1,5,5,3,5,5,5,5,5,4,...,1,57,4,287,1,1,1,5500,0,2500
2,2,5,5,5,4,5,4,1,0,1,...,1,31,4,705,1,1,1,4300,0,4000
3,3,4,5,5,6,4,6,3,4,4,...,1,35,6,1423,1,1,1,4100,0,3000
4,4,5,6,4,5,4,5,1,0,1,...,1,36,4,141,1,1,1,2800,0,2000


In [129]:
know_test = [pd.read_csv(path) for path in sorted(glob('../test/*.csv'))]
know_test[0].head() # 2017년도 test 샘플

,idx,aq1_1,aq1_2,aq2_1,aq2_2,aq3_1,aq3_2,aq4_1,aq4_2,aq5_1,...,bq36,bq37,bq38,bq38_1,bq39_1,bq39_2,bq40,bq41_1,bq41_2,bq41_3
0,0,3,4,2,2,3,3,1,,3,...,2,26,3,비서학,1,1,1,3000,,2300
1,1,5,5,3,5,5,5,5,5,4,...,1,57,4,농화학,1,1,1,5500,,2500
2,2,5,5,5,4,5,4,1,,1,...,1,31,4,신문방송,1,1,1,4300,,4000
3,3,4,5,5,6,4,6,3,4,4,...,1,35,6,화학,1,1,1,4100,,3000
4,4,5,6,4,5,4,5,1,,1,...,1,36,4,광고홍보,1,1,1,2800,,2000


In [130]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [131]:
years = ['2017', '2018', '2019', '2020']

for year, df in zip(years, know_test):
    print(year)
    encoders = {}
    
    for col in df.columns:
        
        try:
            df[col] = df[col].map(int)
        except:
            encoder = year_encoder[year][col]
            df[col] = df[col].map(str)
            category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
            df[col] = df[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
            

2017
2018
2019
2020


In [132]:
test_data = {}
for year, df in zip(years, know_test):
    test_data[year] =  {'X': df.iloc[:,1:]}

In [134]:
rf_predicts = [] 

for year in tqdm(years):
    pred = rf_models[year].predict(test_data[year]['X'])
    rf_predicts.extend(pred)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.70s/it]


In [136]:
submission = pd.read_csv('../submission/sample_submission.csv') # sample submission 불러오기

In [138]:
submission['knowcode'] = rf_predicts

submission.to_csv('../submission/baseline_submission.csv', index=False)